In [6]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
import wikipedia
from lxml import html
import os
import re
import getpass
from google import genai
headers = {
    "User-Agent": "Mozilla/5.0"
}
BASE_URL = "https://en.wikipedia.org/wiki/"

In [7]:
def search_wikipedia(movie_name,year):
    search_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": f"{movie_name}  {year}",
        "format": "json"
    }

    r = requests.get(search_url, params=params, headers=headers,timeout=20)
    data = r.json()

    if data["query"]["search"]:
        return data["query"]["search"][0]["pageid"]
    else:
        return None


# --------------------------
# Get plot from page
# --------------------------
def get_plot_from_pageid(pageid):
    if pageid !=  None:
        try:
            page = wikipedia.WikipediaPage(pageid=pageid)
            text = page.section(section_title="Plot")
            return text
        except wikipedia.DisambiguationError as e:
            print("error occured",e)
            return ""
    else:
        print("No page exists for pageid",pageid)
        return ""









In [8]:
import pandas as pd
import time
import os

df_cast = pd.read_csv("malayalam_movie_cast_dataset.csv")
df_movies = df_cast[["movie_name", "year"]].drop_duplicates()

output_file = "movies_with_plot.csv"

if not os.path.exists(output_file):
    pd.DataFrame(columns=["movie_name", "year", "plot"]).to_csv(output_file, index=False)

done = set(pd.read_csv(output_file)["movie_name"])

for _, row in df_movies.iterrows():
    movie = row["movie_name"]
    year = row["year"]

    if movie in done:
        continue

    print(f"Processing: {movie}")

    try:
        page_id = search_wikipedia(movie, year)   # must use timeout inside
        plot = get_plot_from_pageid(page_id)      # must use timeout inside

    except Exception as e:
        print("Skipping", movie, "due to", e)
        plot = ""

    pd.DataFrame([{
        "movie_name": movie,
        "year": year,
        "plot": plot
    }]).to_csv(output_file, mode="a", header=False, index=False)

    time.sleep(1)

print("✅ Done")


Processing: Marupadi
Processing: January
Processing: April
Processing: Fukri
Processing: Oru Mexican Aparatha
Processing: C/O Saira Banu
Processing: Pareeth Pandari
Processing: Take Off
Processing: Kambhoji
Processing: Rakshadhikari Baiju Oppu
Processing: Vedham
Processing: Adventures of Omanakuttan
Processing: Godha
Processing: Gold Coins
Processing: Ma Chu Ka
Processing: Avarude Raavukal
Processing: Viswasapoorvam Mansoor
Processing: Thondimuthalum Driksakshiyum
Processing: Tiyaan
Processing: Sunday Holiday
Processing: Minnaminungu
Processing: Team 5
Processing: Theeram
Processing: Sarvopari Palakkaran
Processing: Thrissivaperoor Kliptham
Processing: E
Processing: Honey Bee 2.5
Processing: Naval Enna Jewel
Processing: Njandukalude Nattil Oridavela
Processing: Pullikkaran Staraa
Processing: Matchbox
Processing: Udaharanam Sujatha
Processing: Tharangam
Processing: Kaattu
Processing: Vishwa Vikhyatharaya Payyanmar
Processing: Zacharia Pothen Jeevichirippundu
Processing: Paathi
Processin